In [2]:
module Lilly1966_solver

using LinearAlgebra

export full_EQ_set, OSE_set

function full_EQ_set( α_in, ε_in, Re_in, N_in )
# Solve the complete set of the equations (2.25)-(2.28) in Lilly (1966)
    #-- arguments input:
    # α_in: wavenumber for y-direction [non-dim,Vector{Float64}]
    # ε_in: flow angle for Vg [degree,Vector{Float64}]
    # Re_in: Reinolds number [non-dim,Vector{Float64}]
    # N_in: grid numbers in vertical [1,Int64]
    #-- arguments output:
    # c_out: wave speed [non-dim,Array{Complex{Float64},size(α),size(ε),size(Re)}]

    # set parameters
    d2r = π / 180.0  # degree to radian
    na = size(α_in)[1]
    ne = size(ε_in)[1]
    nr = size(Re_in)[1]
    N = N_in

    # initialize variables and matrix
    ntot = 2*N-1  # matrix dimension
    c_out = reshape(zeros(Complex{Float64},na,ne,nr),na,ne,nr)

    # initialize working arrays in this function
    ### --- From here, supposing matrix element as row, col as in mathematical notation
    A = reshape(zeros(Complex{Float64},ntot,ntot),ntot,ntot)  # linear operator in differential equations
    B = reshape(zeros(Complex{Float64},ntot,ntot),ntot,ntot)  # linear operator in differential equations
    A_φφ = reshape(zeros(Complex{Float64},N-1,N-1),N-1,N-1)  # partial matrix (phi operator on phi EQ.) in A
    A_φμ = reshape(zeros(Complex{Float64},N-1,N),N-1,N)      # partial matrix (mu operator on phi EQ.) in A
    A_μφ = reshape(zeros(Complex{Float64},N,N-1),N,N-1)      # partial matrix (phi operator on mu EQ.) in A
    A_μμ = reshape(zeros(Complex{Float64},N,N),N,N)          # partial matrix (mu operator on mu EQ.) in A
    B_φφ = reshape(zeros(Complex{Float64},N-1,N-1),N-1,N-1)  # partial matrix (phi operator on phi EQ.) in B
    B_φμ = reshape(zeros(Complex{Float64},N-1,N),N-1,N)      # partial matrix (mu operator on phi EQ.) in B == 0
    B_μφ = reshape(zeros(Complex{Float64},N,N-1),N,N-1)      # partial matrix (phi operator on mu EQ.) in B == 0
    B_μμ = reshape(zeros(Complex{Float64},N,N),N,N)          # partial matrix (mu operator on mu EQ.) in B
    ### --- From here, supposing matrix element as col, row as in julia notation (only using these arrays for matrix calc)
    A_J = reshape(zeros(Complex{Float64},ntot,ntot),ntot,ntot)  # linear operator in differential equations
    B_J = reshape(zeros(Complex{Float64},ntot,ntot),ntot,ntot)  # linear operator in differential equations

    U_bar = reshape(zeros(N,1),N,1)          # Mean flow in x-direction defined at i+1/2
    V_bar = reshape(zeros(N-1,1),N-1,1)      # Mean flow in y-direction defined at i
    Vh_bar = reshape(zeros(N,1),N,1)      # Mean flow in y-direction defined at i+1/2
    dUdz  = reshape(zeros(N,1),N,1)          # Mean flow shear in x-direction defined at i+1/2
    dVdz  = reshape(zeros(N-1,1),N-1,1)      # Mean flow shear in y-direction defined at i
    d2Vdz2  = reshape(zeros(N-1,1),N-1,1)      # Mean flow shear in y-direction defined at i

    #-- Start loops for varying parameters (α, ε, Re)
    for k in 1:nr
        for j in 1:ne
            for i in 1:na
                α = α_in[i]
                ε = ε_in[j] * d2r
                Re = Re_in[k]
                Δ = 1.0/sqrt(α * N)  # grid spacing
                
                # set grid spacing and mean-wind profiles in vertical
                z_p   = [i*Δ for i in 1:N-1]  # non-dimensional vertical coordinate for φ
                zh_p  = [(i-0.5)*Δ for i in 1:N]  # non-dimensional vertical coordinate for μ
                z_p = reshape(z_p,N-1,1)
                zh_p = reshape(zh_p,N,1)
                
                #-- From here, considering to move them as external argument
                U_bar[1:N,1] .= cos(ε) .- exp.(-zh_p[1:N,1]) .* cos.(zh_p[1:N,1] .+ ε)
                V_bar[1:N-1,1] .= -sin(ε) .+ exp.(-z_p[1:N-1,1]) .* sin.(z_p[1:N-1,1] .+ ε)
                Vh_bar[1:N,1] .= -sin(ε) .+ exp.(-zh_p[1:N,1]) .* sin.(zh_p[1:N,1] .+ ε)
                dUdz[1:N,1] .= exp.(-zh_p[1:N,1]) .* cos.(zh_p[1:N,1] .+ ε) .+ exp.(-zh_p[1:N,1]) .* sin.(zh_p[1:N,1] .+ ε)
                dVdz[1:N-1,1] .= -exp.(-z_p[1:N-1,1]) .* sin.(z_p[1:N-1,1] .+ ε) .+ exp.(-z_p[1:N-1,1]) .* cos.(z_p[1:N-1,1] .+ ε)
                d2Vdz2[1:N-1,1] .= .- 2.0 .* exp.(-z_p[1:N-1,1]) .* cos.(z_p[1:N-1,1] .+ ε)

                # setting matrix elements in A and B
                Δ2 = Δ*Δ
                Δ4 = Δ2*Δ2
                αΔ = α*Δ
                V_infl = -sin(ε) + exp(-(0.5*π-ε))
                #      μ[1]        μ[2]       μ[3]        μ[4]
                # ----- 0 --------- 1 --------- 2 --------- 3 -- i for μ = μ[i+1]
                # ------|-----------|-----------|-----------|---   for μ = μ[i+1]
                # 0---------- 1 --------- 2 --------- 3 -------- i for φ = φ[i]
                # |-----------|-----------|-----------|---------   for φ = φ[i]

                # For φ EQ.
                for i in 3:N-3
                    #    println("$i")
                    A_φφ[i,i+2] = complex(1.0,α*Re*V_bar[i,1]*Δ2/12.0)
                    A_φφ[i,i-2] = A_φφ[i,i+2]
                    A_φφ[i,i+1] = complex(-4.0-2.0*((αΔ)^2),-4.0*α*Re*V_bar[i,1]*Δ2/3.0)
                    A_φφ[i,i-1] = A_φφ[i,i+1]
                    A_φφ[i,i]   = complex(6.0+4.0*((αΔ)^2)+(α^4-d2Vdz2[i,1])*Δ4,α*Re*V_bar[i,1]*(2.5+(αΔ)^2)*Δ2)
                    A_φμ[i,i]   = -2.0*(Δ^3)  # A^{φ,μ}_{i,i-1/2}
                    A_φμ[i,i+1] = +2.0*(Δ^3)  # A^{φ,μ}_{i,i+1/2}
                    B_φφ[i,i+2] = complex(0.0,α*Re*Δ2/12.0)
                    B_φφ[i,i-2] = B_φφ[i,i+2]
                    B_φφ[i,i+1] = -complex(0.0,4.0*α*Re*Δ2/3.0)
                    B_φφ[i,i-1] = B_φφ[i,i+1]
                    B_φφ[i,i]   = complex(0.0,α*Re*(2.5+(αΔ)^2)*Δ2)
                end

                # For μ EQ.
                for i in 3:N-2
                    A_μφ[i,i] = -complex(2.0*Δ,9.0*α*Re*dUdz[i,1]*Δ2/16.0)  # A^{μ,φ}_{i+1/2,i+1}
                    A_μφ[i,i-2] = complex(0.0,α*Re*dUdz[i,1]*Δ2/16.0)  # A^{μ,φ}_{i+1/2,i-1}
                    A_μφ[i,i+1] = A_μφ[i,i-2]  # A^{μ,φ}_{i+1/2,i+2}
                    A_μφ[i,i-1] = complex(2.0*Δ,-9.0*α*Re*dUdz[i,1]*Δ2/16.0)  # A^{μ,φ}_{i+1/2,i}
                    A_μμ[i,i+1] = 1.0  # A^{μ,μ}_{i+1/2,i-1/2}
                    A_μμ[i,i-1] = 1.0  # A^{μ,μ}_{i+1/2,i+3/2}
                    A_μμ[i,i]   = -complex(2.0+(αΔ)^2,α*Re*Vh_bar[i,1]*Δ2)
                    B_μμ[i,i]   = -complex(0.0,α*Re*Δ2)
                end

                # Adjacent of boundaries
                # For φ EQ. (at z=z_1)
                A_φφ[1,3] = complex(1.0,α*Re*V_bar[1,1]*Δ2/12.0)
                A_φφ[1,2] = complex(-4.0-2.0*((αΔ)^2),-4.0*α*Re*V_bar[1,1]*Δ2/3.0)
                A_φφ[1,1] = complex(7.0+4.0*((αΔ)^2)+(α^4-d2Vdz2[1,1])*Δ4,α*Re*V_bar[1,1]*(31.0/12.0+(αΔ)^2)*Δ2)
                A_φμ[1,1] = -2.0*(Δ^3)
                A_φμ[1,2] = +2.0*(Δ^3)
                B_φφ[1,3] = complex(0.0,α*Re*Δ2/12.0)
                B_φφ[1,2] = -complex(0.0,4.0*α*Re*Δ2/3.0)
                B_φφ[1,1] = complex(0.0,α*Re*(31.0/12.0+(αΔ)^2)*Δ2)
                # For μ EQ. (at z=z_{1/2})
                A_μφ[1,1] = -complex(2.0*Δ,0.5*α*Re*dUdz[1,1]*Δ2)
                A_μφ[1,2] = complex(0.0,α*Re*dUdz[1,1]*Δ2/16.0)
                A_μμ[1,2] = 1.0
                A_μμ[1,1] = -complex(3.0+(αΔ)^2,α*Re*Vh_bar[1,1]*Δ2)
                B_μμ[1,1] = -complex(0.0,α*Re*Δ2)

                # For φ EQ. (at z=z_2)
                A_φφ[2,4] = complex(1.0,α*Re*V_bar[2,1]*Δ2/12.0)
                A_φφ[2,3] = complex(-4.0-2.0*((αΔ)^2),-4.0*α*Re*V_bar[2,1]*Δ2/3.0)
                A_φφ[2,1] = A_φφ[2,3]
                A_φφ[2,2] = complex(6.0+4.0*((αΔ)^2)+(α^4-d2Vdz2[2,1])*Δ4,α*Re*V_bar[2,1]*(2.5+(αΔ)^2)*Δ2)
                A_φμ[2,2] = -2.0*(Δ^3)
                A_φμ[2,3] = +2.0*(Δ^3)
                B_φφ[2,4] = complex(0.0,α*Re*Δ2/12.0)
                B_φφ[2,3] = -complex(0.0,4.0*α*Re*Δ2/3.0)
                B_φφ[2,1] = B_φφ[2,3]
                B_φφ[2,2] = complex(0.0,α*Re*(2.5+(αΔ)^2)*Δ2)
                # For μ EQ. (at z=z_{1+1/2})
                A_μφ[2,2] = -complex(2.0*Δ,9.0*α*Re*dUdz[2,1]*Δ2/16.0)
                A_μφ[2,3] = complex(0.0,α*Re*dUdz[2,1]*Δ2/16.0)
                A_μφ[2,1] = complex(2.0*Δ,-9.0*α*Re*dUdz[2,1]*Δ2/16.0)
                A_μμ[2,3] = 1.0
                A_μμ[2,1] = 1.0
                A_μμ[2,2] = -complex(2.0+(αΔ)^2,α*Re*Vh_bar[2,1]*Δ2)
                B_μμ[2,2] = -complex(0.0,α*Re*Δ2)

                # For φ EQ. (at z=z_{N-2})
                A_φφ[N-2,N-4] = complex(1.0,α*Re*V_bar[N-2,1]*Δ2/12.0)
                A_φφ[N-2,N-1] = complex(-4.0-2.0*((αΔ)^2),-4.0*α*Re*V_bar[N-2,1]*Δ2/3.0)
                A_φφ[N-2,N-3] = A_φφ[N-2,N-1]
                A_φφ[N-2,N-2] = complex(6.0+4.0*((αΔ)^2)+(α^4-d2Vdz2[N-2,1])*Δ4,α*Re*V_bar[N-2,1]*(2.5+(αΔ)^2)*Δ2)
                A_φμ[N-2,N-2] = -2.0*(Δ^3)
                A_φμ[N-2,N-1] = +2.0*(Δ^3)
                B_φφ[N-2,N-4] = complex(0.0,α*Re*Δ2/12.0)
                B_φφ[N-2,N-1] = -complex(0.0,4.0*α*Re*Δ2/3.0)
                B_φφ[N-2,N-3] = B_φφ[N-2,N-1]
                B_φφ[N-2,N-2] = complex(0.0,α*Re*(2.5+(αΔ)^2)*Δ2)
                # For μ EQ. (at z=z_{N-1-1/2})
                A_μφ[N-1,N-1] = -complex(2.0*Δ,9.0*α*Re*dUdz[N-1,1]*Δ2/16.0)
                A_μφ[N-1,N-3] = complex(0.0,α*Re*dUdz[N-1,1]*Δ2/16.0)
                A_μφ[N-1,N-2] = complex(2.0*Δ,-9.0*α*Re*dUdz[N-1,1]*Δ2/16.0)
                A_μμ[N-1,N-2] = 1.0
                A_μμ[N-1,N]   = 1.0
                A_μμ[N-1,N-1] = -complex(2.0+(αΔ)^2,α*Re*Vh_bar[N-1,1]*Δ2)
                B_μμ[N-1,N-1] = -complex(0.0,α*Re*Δ2)

                # For φ EQ. (at z=z_{N-1})
                A_φφ[N-1,N-3] = complex(1.0,α*Re*V_bar[N-1,1]*Δ2/12.0)
                A_φφ[N-1,N-2] = complex(-4.0-2.0*((αΔ)^2),-4.0*α*Re*V_bar[N-1,1]*Δ2/3.0)
                A_φφ[N-1,N-1] = complex(5.0+4.0*((αΔ)^2)+(α^4-d2Vdz2[N-1,1])*Δ4,α*Re*V_bar[N-1,1]*(29.0/12.0+(αΔ)^2)*Δ2)
                A_φμ[N-1,N-1] = -2.0*(Δ^3)
                A_φμ[N-1,N]   = +2.0*(Δ^3)
                B_φφ[N-1,N-3] = complex(0.0,α*Re*Δ2/12.0)
                B_φφ[N-1,N-2] = -complex(0.0,4.0*α*Re*Δ2/3.0)
                B_φφ[N-1,N-1] = complex(0.0,α*Re*(29.0/12.0+(αΔ)^2)*Δ2)
                # For μ EQ. (at z=z_{N-1/2})
                A_μφ[N,N-2]   = complex(0.0,α*Re*dUdz[N]*Δ2/16.0)
                A_μφ[N,N-1]   = complex(2.0*Δ,-5.0*α*Re*dUdz[N]*Δ2/8.0)
                A_μμ[N,N-1]   = 1.0
                A_μμ[N,N]     = -complex(1.0+(αΔ)^2,α*Re*Vh_bar[N]*Δ2)
                B_μμ[N,N]     = -complex(0.0,α*Re*Δ2)

                # assigned partial matrices A^ and B^ to A and B
                A[1:N-1,1:N-1] = A_φφ[1:N-1,1:N-1]
                A[1:N-1,N:2*N-1] = A_φμ[1:N-1,1:N]
                A[N:2*N-1,1:N-1] = A_μφ[1:N,1:N-1]
                A[N:2*N-1,N:2*N-1] = A_μμ[1:N,1:N]
                # non-diagonal parts in B are zero
                B[1:N-1,1:N-1] = B_φφ[1:N-1,1:N-1]
                B[N:2*N-1,N:2*N-1] = B_μμ[1:N,1:N]

                A_J = transpose(A)  # mathematical A -> Julia A_J
                B_J = transpose(B)  # mathematical A -> Julia A_J
                C_J = inv(B_J) * A_J
                D_J = eigvals(C_J)
                F_J = imag.(D_J)
                E_J = inv(B_J) * B_J
                nc_max = findmax(F_J)[2]
                c_out[i,j,k] = D_J[nc_max]

            end
        end
    end

#println(D_J[findmax(F_J)[2]])
#println(F_J)
#println(V_infl)

    return c_out

end  # function full_EQ_set
    
function OSE_set( α_in, ε_in, Re_in, N_in )
# Solve the approximation (OSE) set of the equations (2.25), (2.27)-(2.28) in Lilly (1966)
    #-- arguments input:
    # α_in: wavenumber for y-direction [non-dim,Vector{Float64}]
    # ε_in: flow angle for Vg [degree,Vector{Float64}]
    # Re_in: Reinolds number [non-dim,Vector{Float64}]
    # N_in: grid numbers in vertical [1,Int64]
    #-- arguments output:
    # c_out: wave speed [non-dim,Array{Complex{Float64},size(α),size(ε),size(Re)}]

    # set parameters
    d2r = π / 180.0  # degree to radian
    na = size(α_in)[1]
    ne = size(ε_in)[1]
    nr = size(Re_in)[1]
    N = N_in

    # initialize variables and matrix
    ntot = N-1  # matrix dimension
    c_out = reshape(zeros(Complex{Float64},(na,ne,nr)),na,ne,nr)

    # initialize working arrays in this function
    ### --- From here, supposing matrix element as row, col as in mathematical notation
    A = reshape(zeros(Complex{Float64},ntot,ntot),ntot,ntot)  # linear operator in differential equations
    B = reshape(zeros(Complex{Float64},ntot,ntot),ntot,ntot)  # linear operator in differential equations
    A_φφ = reshape(zeros(Complex{Float64},N-1,N-1),N-1,N-1)  # partial matrix (phi operator on phi EQ.) in A
    B_φφ = reshape(zeros(Complex{Float64},N-1,N-1),N-1,N-1)  # partial matrix (phi operator on phi EQ.) in B
    ### --- From here, supposing matrix element as col, row as in julia notation (only using these arrays for matrix calc)
    A_J = reshape(zeros(Complex{Float64},ntot,ntot),ntot,ntot)  # linear operator in differential equations
    B_J = reshape(zeros(Complex{Float64},ntot,ntot),ntot,ntot)  # linear operator in differential equations

    U_bar = reshape(zeros(N,1),N,1)          # Mean flow in x-direction defined at i+1/2
    V_bar = reshape(zeros(N-1,1),N-1,1)      # Mean flow in y-direction defined at i
    Vh_bar = reshape(zeros(N,1),N,1)      # Mean flow in y-direction defined at i+1/2
    dUdz  = reshape(zeros(N,1),N,1)          # Mean flow shear in x-direction defined at i+1/2
    dVdz  = reshape(zeros(N-1,1),N-1,1)      # Mean flow shear in y-direction defined at i
    d2Vdz2  = reshape(zeros(N-1,1),N-1,1)      # Mean flow shear in y-direction defined at i

    #-- Start loops for varying parameters (α, ε, Re)
    for k in 1:nr
        for j in 1:ne
            for i in 1:na
                α = α_in[i]
                ε = ε_in[j] * d2r
                Re = Re_in[k]
                Δ = 1.0/sqrt(α * N)  # grid spacing
                
                # set grid spacing and mean-wind profiles in vertical
                z_p   = [i*Δ for i in 1:N-1]  # non-dimensional vertical coordinate for φ
                zh_p  = [(i-0.5)*Δ for i in 1:N]  # non-dimensional vertical coordinate for μ
                z_p = reshape(z_p,N-1,1)
                zh_p = reshape(zh_p,N,1)
                
                #-- From here, considering to move them as external argument
                U_bar[1:N,1] .= cos(ε) .- exp.(-zh_p[1:N,1]) .* cos.(zh_p[1:N,1] .+ ε)
                V_bar[1:N-1,1] .= -sin(ε) .+ exp.(-z_p[1:N-1,1]) .* sin.(z_p[1:N-1,1] .+ ε)
                Vh_bar[1:N,1] .= -sin(ε) .+ exp.(-zh_p[1:N,1]) .* sin.(zh_p[1:N,1] .+ ε)
                dUdz[1:N,1] .= exp.(-zh_p[1:N,1]) .* cos.(zh_p[1:N,1] .+ ε) .+ exp.(-zh_p[1:N,1]) .* sin.(zh_p[1:N,1] .+ ε)
                dVdz[1:N-1,1] .= -exp.(-z_p[1:N-1,1]) .* sin.(z_p[1:N-1,1] .+ ε) .+ exp.(-z_p[1:N-1,1]) .* cos.(z_p[1:N-1,1] .+ ε)
                d2Vdz2[1:N-1,1] .= .- 2.0 .* exp.(-z_p[1:N-1,1]) .* cos.(z_p[1:N-1,1] .+ ε)

                # setting matrix elements in A and B
                Δ2 = Δ*Δ
                Δ4 = Δ2*Δ2
                αΔ = α*Δ
                V_infl = -sin(ε) + exp(-(0.5*π-ε))
                #      μ[1]        μ[2]       μ[3]        μ[4]
                # ----- 0 --------- 1 --------- 2 --------- 3 -- i for μ = μ[i+1]
                # ------|-----------|-----------|-----------|---   for μ = μ[i+1]
                # 0---------- 1 --------- 2 --------- 3 -------- i for φ = φ[i]
                # |-----------|-----------|-----------|---------   for φ = φ[i]

                # For φ EQ.
                for i in 3:N-3
                    #    println("$i")
                    A_φφ[i,i+2] = complex(1.0,α*Re*V_bar[i,1]*Δ2/12.0)
                    A_φφ[i,i-2] = A_φφ[i,i+2]
                    A_φφ[i,i+1] = complex(-4.0-2.0*((αΔ)^2),-4.0*α*Re*V_bar[i,1]*Δ2/3.0)
                    A_φφ[i,i-1] = A_φφ[i,i+1]
                    A_φφ[i,i]   = complex(6.0+4.0*((αΔ)^2)+(α^4-d2Vdz2[i,1])*Δ4,α*Re*V_bar[i,1]*(2.5+(αΔ)^2)*Δ2)
                    B_φφ[i,i+2] = complex(0.0,α*Re*Δ2/12.0)
                    B_φφ[i,i-2] = B_φφ[i,i+2]
                    B_φφ[i,i+1] = -complex(0.0,4.0*α*Re*Δ2/3.0)
                    B_φφ[i,i-1] = B_φφ[i,i+1]
                    B_φφ[i,i]   = complex(0.0,α*Re*(2.5+(αΔ)^2)*Δ2)
                end

                # Adjacent of boundaries
                # For φ EQ. (at z=z_1)
                A_φφ[1,3] = complex(1.0,α*Re*V_bar[1,1]*Δ2/12.0)
                A_φφ[1,2] = complex(-4.0-2.0*((αΔ)^2),-4.0*α*Re*V_bar[1,1]*Δ2/3.0)
                A_φφ[1,1] = complex(7.0+4.0*((αΔ)^2)+(α^4-d2Vdz2[1,1])*Δ4,α*Re*V_bar[1,1]*(31.0/12.0+(αΔ)^2)*Δ2)
                B_φφ[1,3] = complex(0.0,α*Re*Δ2/12.0)
                B_φφ[1,2] = -complex(0.0,4.0*α*Re*Δ2/3.0)
                B_φφ[1,1] = complex(0.0,α*Re*(31.0/12.0+(αΔ)^2)*Δ2)

                # For φ EQ. (at z=z_2)
                A_φφ[2,4] = complex(1.0,α*Re*V_bar[2,1]*Δ2/12.0)
                A_φφ[2,3] = complex(-4.0-2.0*((αΔ)^2),-4.0*α*Re*V_bar[2,1]*Δ2/3.0)
                A_φφ[2,1] = A_φφ[2,3]
                A_φφ[2,2] = complex(6.0+4.0*((αΔ)^2)+(α^4-d2Vdz2[2,1])*Δ4,α*Re*V_bar[2,1]*(2.5+(αΔ)^2)*Δ2)
                B_φφ[2,4] = complex(0.0,α*Re*Δ2/12.0)
                B_φφ[2,3] = -complex(0.0,4.0*α*Re*Δ2/3.0)
                B_φφ[2,1] = B_φφ[2,3]
                B_φφ[2,2] = complex(0.0,α*Re*(2.5+(αΔ)^2)*Δ2)

                # For φ EQ. (at z=z_{N-2})
                A_φφ[N-2,N-4] = complex(1.0,α*Re*V_bar[N-2,1]*Δ2/12.0)
                A_φφ[N-2,N-1] = complex(-4.0-2.0*((αΔ)^2),-4.0*α*Re*V_bar[N-2,1]*Δ2/3.0)
                A_φφ[N-2,N-3] = A_φφ[N-2,N-1]
                A_φφ[N-2,N-2] = complex(6.0+4.0*((αΔ)^2)+(α^4-d2Vdz2[N-2,1])*Δ4,α*Re*V_bar[N-2,1]*(2.5+(αΔ)^2)*Δ2)
                B_φφ[N-2,N-4] = complex(0.0,α*Re*Δ2/12.0)
                B_φφ[N-2,N-1] = -complex(0.0,4.0*α*Re*Δ2/3.0)
                B_φφ[N-2,N-3] = B_φφ[N-2,N-1]
                B_φφ[N-2,N-2] = complex(0.0,α*Re*(2.5+(αΔ)^2)*Δ2)

                # For φ EQ. (at z=z_{N-1})
                A_φφ[N-1,N-3] = complex(1.0,α*Re*V_bar[N-1,1]*Δ2/12.0)
                A_φφ[N-1,N-2] = complex(-4.0-2.0*((αΔ)^2),-4.0*α*Re*V_bar[N-1,1]*Δ2/3.0)
                A_φφ[N-1,N-1] = complex(5.0+4.0*((αΔ)^2)+(α^4-d2Vdz2[N-1,1])*Δ4,α*Re*V_bar[N-1,1]*(29.0/12.0+(αΔ)^2)*Δ2)
                B_φφ[N-1,N-3] = complex(0.0,α*Re*Δ2/12.0)
                B_φφ[N-1,N-2] = -complex(0.0,4.0*α*Re*Δ2/3.0)
                B_φφ[N-1,N-1] = complex(0.0,α*Re*(29.0/12.0+(αΔ)^2)*Δ2)

                # assigned partial matrices A^ and B^ to A and B
                A[1:N-1,1:N-1] = A_φφ[1:N-1,1:N-1]
                # non-diagonal parts in B are zero
                B[1:N-1,1:N-1] = B_φφ[1:N-1,1:N-1]

                A_J = transpose(A)  # mathematical A -> Julia A_J
                B_J = transpose(B)  # mathematical A -> Julia A_J
                C_J = inv(B_J) * A_J
                D_J = eigvals(C_J)
                F_J = imag.(D_J)
                E_J = inv(B_J) * B_J
                nc_max = findmax(F_J)[2]
                c_out[i,j,k] = D_J[nc_max]

            end
        end
    end

#println(D_J[findmax(F_J)[2]])
#println(F_J)
#println(V_infl)

    return c_out

end  # function OSE_set
    
end  # module Lilly1966_solver

Main.Lilly1966_solver

In [3]:
using .Lilly1966_solver

#-- Drawing parameters    
na = 20  # wavenumber interval
ne = 20  # angle interval
nr = 20  # Reinolds number interval
α_min = 0.001
α_max = 1.0
ε_min = -60.0
ε_max = 60.0
Re_min = 25.0
Re_max = 500.0

α = [α_min+(α_max-α_min)*((i-1.0)/(na-1.0)) for i in 1:na]
ε = [ε_min+(ε_max-ε_min)*((i-1.0)/(ne-1.0)) for i in 1:ne]
Re = [Re_min+(Re_max-Re_min)*((i-1.0)/(nr-1.0)) for i in 1:nr]

c_full = full_EQ_set( α, ε, Re, 35 )
c_ose = OSE_set( α, ε, Re, 35 )



20×20×20 Array{ComplexF64, 3}:
[:, :, 1] =
 0.864005+0.749363im   0.804944+0.14258im    …  -0.866025-0.0236016im
 0.864131-0.0178902im  0.803465-0.0170507im     -0.869369-0.0216414im
 0.863543-0.0192996im  0.802856-0.0185836im     -0.871508-0.0239824im
 0.863595-0.0210307im  0.802932-0.0203377im     -0.873849-0.0264434im
 0.864056-0.0230905im   0.80343-0.022402im      -0.876698-0.0286047im
  0.86475-0.0254921im  0.804155-0.024811im   …  -0.879753-0.0293886im
 0.865561-0.0282094im  0.804976-0.0275412im     -0.881353-0.0290929im
 0.866423-0.031197im   0.805824-0.0305438im     -0.881612-0.0292257im
  0.86731-0.0344051im  0.806668-0.0337645im     -0.881312-0.0301184im
 0.868215-0.037786im   0.807505-0.0371523im     -0.880885-0.0316704im
 0.869146-0.0412972im  0.808344-0.0406621im  …  -0.880543-0.0337225im
 0.870114-0.044902im   0.809203-0.0442561im     -0.880385-0.0361257im
 0.871132-0.0485695im  0.810098-0.0479034im     -0.880442-0.0387554im
 0.872215-0.0522742im  0.811049-0.051579im     

In [6]:
#-- Drawing
##########
#  Plot  #
##########
using PyPlot

rc("font", family="IPAPGothic")
fig = figure("pyplot_majorminor",figsize=(7,5))

nx = na
ny = ne

xax = reshape(zeros(nx,ny),nx,ny)
yax = reshape(zeros(nx,ny),nx,ny)

for i in 1:nx
    xax[i,1:ny] .= α[i]  # xmin + (xmax-xmin) * (i-1) / (nx-1)
end
for i in 1:ny
    yax[1:nx,i] .= ε[i]  # ymin + (ymax-ymin) * (i-1) / (ny-1)
#    yax[1:nx,i] .= Re[i]  # ymin + (ymax-ymin) * (i-1) / (ny-1)
end

cont_val = reshape(zeros(nx,ny),nx,ny)
shade_val = reshape(zeros(nx,ny),nx,ny)

for i in 1:nr#div(nr,2)-1:div(nr,2)+1
    cont_val = real.(c_full[1:na,1:ne,i])
    shade_val = imag.(c_full[1:na,1:ne,i])
#draw_num2 = 2
#if draw_num2 == 1
#    cp = contourf(xax[1:nx,1:nx], xax'[1:nx,1:nx], Pftime[1:nx,1:nx,1], levels=[-15.0, -10.0, -5.0, 0.0, 5.0, 10.0, 15.0, 20.0], origin="image", cmap=ColorMap("viridis"), extend="both")
    #cp = contourf(xax[1:nx,1:nx], xax'[1:nx,1:nx], Pftime[1:nx,1:nx,2], levels=[-2.0, -1.0, -0.5, -0.25, 0.25, 0.5, 1.0, 1.5, 2.0], origin="image", cmap=ColorMap("viridis"), extend="both")
    #cp = contourf(xax[1:nx,1:no], xax'[1:nx,1:no], Pftime[1:nx,1:nx,2]*Hop'*inv(Hop*Pftime[1:nx,1:nx,2]*Hop'+Ro), origin="image", cmap=ColorMap("viridis"), extend="both")
#elseif draw_num2 == 2
    ##cp = contour(xax[1:nx,1:ny], yax[1:nx,1:ny], shade_val[1:nx,1:ny], 8, levels=[-0.01, -0.005, -0.003, -0.002, -0.001, -0.0005, 0.0, 0.0005, 0.001, 0.002, 0.003, 0.005, 0.01])
    cp = contourf(xax[1:nx,1:ny], yax[1:nx,1:ny], shade_val[1:nx,1:ny], 13, levels=[-0.02, -0.01, -0.005, -0.0025, -0.001, -0.0005, 0.0, 0.0005, 0.001, 0.0025, 0.005, 0.01, 0.02], cmap=ColorMap("viridis"), extend="both")
    ##cp = contourf(xax[1:nx,1:ny], yax[1:nx,1:ny], cont_val[1:nx,1:ny], 8, levels=[-1.0, -0.5, -0.2, -0.1, 0.0, 0.1, 0.2, 0.5, 1.0], cmap=ColorMap("viridis"), extend="both")
    #cp = contourf(xax[1:nx,1:ny], yax[1:nx,1:ny], shade_val[1:nx,1:ny], 10, cmap=ColorMap("viridis"), extend="both")
#elseif draw_num2 == 3
#    cp = contourf(xoax[1:nx,1:nto-1], toax[1:nx,1:nto-1], evec_max[1:nx,1:nto-1], 10, cmap=ColorMap("viridis"), extend="both")
#elseif draw_num2 == 4
#    cp = contourf(xoax[1:nx,1:40], toax[1:nx,1:40], x_inc[1:nx,1:40], levels=[-2.0, -1.0, -0.5, -0.25, 0.25, 0.5, 1.0, 1.5, 2.0], cmap=ColorMap("viridis"), extend="both")
#elseif draw_num2 == 5
#    cp = contourf(xoax[1:no,1:40], toax[1:no,1:40], y_innov[1:no,1:40], levels=[-2.0, -1.0, -0.5, -0.25, 0.25, 0.5, 1.0, 1.5, 2.0], cmap=ColorMap("viridis"), extend="both")
#end
#ax.label(cp, inline=1, fontsize=10)
#legend()
ax = gca()

xlabel("X")
ylabel("Y")
if i == 1 #div(ne,2)-1
   plt.colorbar(cp)
end
grid("on")

PyPlot.title("Re = Re[i]")

#########################
#  Set tick dimensions  #
#########################
#ax.xaxis.set_tick_params(which="major",length=5,width=2,labelsize=10)
#ax.xaxis.set_tick_params(which="minor",length=5,width=2)

fig.canvas.draw() # Update the figure
gcf() # Needed for IJulia to plot inline
fname = "Re" * string(i) * ".png"
#savefig("B-init_2d.pdf")
savefig(fname)
#psi
end
println(shade_val[1:na,1])

Figure(PyObject <Figure size 700x500 with 2 Axes>)

[-0.001584429349547998, 0.02965700876241184, 0.021597269839385477, 0.019761633489755976, 0.016001681302057036, 0.013340835947234772, 0.011126923565145386, 0.009145355882963087, 0.0075119491883417705, 0.006112137941604234, 0.0048805400566256435, 0.003799799285417109, 0.0028450698743907247, 0.0019910040528129925, 0.0012210191935090077, 0.0005236712496438252, -0.00011100587241823574, -0.0006917332699431332, -0.0012255845072084743, -0.0017182414228829911]


In [22]:
imag.(c_full[10,1:ne,1])

20-element Array{Float64,1}:
 -0.00834951244948747
 -0.010847702797636218
 -0.00828769970324535
 -0.010865878572275754
 -0.008264488528986515
 -0.01092594228751652
 -0.00828501393809018
 -0.01103425584827447
 -0.008354616112039475
 -0.011198327640146102
 -0.008478378092726767
 -0.011427010147025988
 -0.008660115429296702
 -0.011730627810989621
 -0.008900263110032685
 -0.01212078621178917
 -0.009191689690546033
 -0.012609081618072917
 -0.009512379222217537
 -0.01306649868664534